# Data Processing Template

Since almost without exception the first step is working with data munging, this notebook is stubbed out with a very select few imports to speed up the initial steps following creating a project and copying in raw data.

In [1]:
# minimal module imports
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayer
from arcgis.geometry import Geometry
import pandas as pd
import os
import importlib

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [46]:
# a couple of handy variables and settings to get started

data = r'../data'
data_raw = os.path.join(data, 'raw')
data_ext = os.path.join(data, 'external')
data_int = os.path.join(data, 'interim')
data_out = os.path.join(data, 'processed')

gdb_int = os.path.join(data_int, 'interim.gdb')
gdb_raw = os.path.join(data_raw, 'raw.gdb')

In [3]:
bp_pth = os.path.join(gdb_raw, 'block_points')
bg_pth = os.path.join(gdb_raw, 'block_groups')
cbsa_pth = os.path.join(gdb_raw, 'cbsas')

arcpy.env.overwriteOutput = True

In [5]:
bg_id_lst = [bg_id for (bg_id,) in arcpy.da.SearchCursor(bg_pth, 'ID')]
bg_id_lst[:5]

['410030103003',
 '410110001001',
 '410110001002',
 '410110001003',
 '410110001004']

In [9]:
bp_id_lst = [bp_geom for (bp_geom,) in arcpy.da.SearchCursor(bp_pth, 'FIPS')]
bp_id_lst[:5]

['150039812001004',
 '150039812001005',
 '150039812001006',
 '150039812001007',
 '150039812001008']

In [14]:
bg_id_len = len(bg_id_lst[0])

12

In [18]:
unique_truncated_id_lst = list(set(bp_id[:bg_id_len] for bp_id in bp_id_lst))
unique_truncated_id_lst[:5]

['482139503001',
 '401350304012',
 '132270501001',
 '551239602002',
 '450510602031']

In [21]:
status_lst = [unique_truncated_id in bg_id_lst for unique_truncated_id in unique_truncated_id_lst]

In [22]:
any(status_lst)

True

In [23]:
bp_id_df = GeoAccessor.from_featureclass(bp_pth)
bp_id_df.head()

,OBJECTID,FIPS,STATE_NAME,SHAPE
0,1,150039812001004,Hawaii,"{""x"": -166.28822369699998, ""y"": 23.86846842700..."
1,2,150039812001005,Hawaii,"{""x"": -166.21047832799997, ""y"": 23.78768193900..."
2,3,150039812001006,Hawaii,"{""x"": -166.24250323499996, ""y"": 23.87162798700..."
3,4,150039812001007,Hawaii,"{""x"": -166.24162861199997, ""y"": 23.87150556400..."
4,5,150039812001008,Hawaii,"{""x"": -166.28719940299996, ""y"": 23.86874607300..."


In [27]:
bp_id_df = bg_id_df
    
bp_id_df['bg_id'] = bp_id_df.FIPS.apply(lambda val: val[:12])

In [47]:
bp_id_df.spatial.to_featureclass(os.path.join(gdb_int, 'block_points_bg_id'))

'D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\block_points_bg_id'

In [48]:
bp_id_df.columns

Index(['OBJECTID', 'FIPS', 'STATE_NAME', 'SHAPE', 'bg_id'], dtype='object')

In [49]:
arcpy.stats.MeanCenter(os.path.join(gdb_int, 'block_points_bg_id'), os.path.join(gdb_int, 'block_group_pop_centroid'), Case_Field='bg_id')

<Result ''>